In [ ]:
import pandas as pd

In [ ]:
loan=pd.read_csv('loan.csv',low_memory=False)

In [ ]:
total_num = loan.isnull().sum().sort_values(ascending=False)

perc = loan.isnull().sum()/loan.isnull().count() *100
perc1 = (round(perc,2).sort_values(ascending=False))

# Creating a data frame:
df_miss = pd.concat([total_num, perc1], axis =1 , keys =["Total Missing Values", "Percentage %"]).sort_values(by ="Percentage %", ascending = False)
top_mis = df_miss[df_miss["Percentage %"]>0]
top_mis.reset_index(inplace=True)

loan.drop(top_mis[top_mis["Percentage %"]>75]["index"], axis = 1, inplace=True)
loan.drop(["next_pymnt_d","mths_since_last_delinq"]
        , axis = 1, inplace=True)
loan.drop(["last_pymnt_d", "last_credit_pull_d", "earliest_cr_line", "issue_d", "addr_state"], axis =1 , inplace = True)
loan["tot_cur_bal"] = loan["tot_cur_bal"].fillna(loan["tot_cur_bal"].mean())

loan["tot_coll_amt"] = loan["tot_coll_amt"].fillna(0)

loan["emp_length"]=loan["emp_length"].fillna(0)

loan["revol_util"]= loan["revol_util"].fillna(loan["revol_util"].mean())

loan["collections_12_mths_ex_med"]= loan["collections_12_mths_ex_med"].fillna(loan["collections_12_mths_ex_med"].mean())

loan["acc_now_delinq"]=loan["acc_now_delinq"].fillna(loan["acc_now_delinq"].mode())

loan["delinq_2yrs"]=loan["delinq_2yrs"].fillna(0)

loan["inq_last_6mths"]= loan["inq_last_6mths"].fillna(0)

loan["pub_rec"]= loan["pub_rec"].fillna(0)

loan["acc_now_delinq"]= loan["acc_now_delinq"].fillna(0)

loan["open_acc"]= loan["open_acc"].fillna(loan["open_acc"].mean())

In [ ]:
loan.dropna(axis=0,inplace=True)

In [ ]:
# Determining the loans that are bad from loan_status column
import numpy as np
bad_loan = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "In Grace Period", 
            "Late (16-30 days)", "Late (31-120 days)"]

# Defining a new column loan_condition to qualify good/ bad loans
loan['loan_condition'] = np.nan


def loan_condition(status):
    if status in bad_loan:
        return 'Bad Loan'
    else:
        return 'Good Loan'
    
#calling function using data frame  
loan['loan_condition'] = loan['loan_status'].apply(loan_condition)

In [ ]:
# Let's create categories for annual_income since most of the bad loans are located below 100k
import numpy as np
loan['income_category'] = np.nan
lst = [loan]

for col in lst:
    col.loc[col['annual_inc'] <= 100000, 'income_category'] = 'Low'
    col.loc[(col['annual_inc'] > 100000) & (col['annual_inc'] <= 200000), 'income_category'] = 'Medium'
    col.loc[col['annual_inc'] > 200000, 'income_category'] = 'High' 

In [ ]:
loan['loan_condition_int'] = np.nan

In [ ]:
# Converting loan_condition (Good / Bad) into (1 / 0) integer

loan['loan_condition_int'] = np.nan
def to_int1(condition):
    if condition == 'Good Loan':
        return 1
    elif condition == 'Bad Loan':
        return 0

loan['loan_condition_int'] = loan['loan_condition'].apply(to_int1)

In [ ]:
employment_length = ['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
                    '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a']

# Create a new column and convert emp_length to integers.

lst = [loan]
loan['emp_length_int'] = np.nan

for col in lst:
    col.loc[col['emp_length'] == '10+ years', "emp_length_int"] = 10
    col.loc[col['emp_length'] == '9 years', "emp_length_int"] = 9
    col.loc[col['emp_length'] == '8 years', "emp_length_int"] = 8
    col.loc[col['emp_length'] == '7 years', "emp_length_int"] = 7
    col.loc[col['emp_length'] == '6 years', "emp_length_int"] = 6
    col.loc[col['emp_length'] == '5 years', "emp_length_int"] = 5
    col.loc[col['emp_length'] == '4 years', "emp_length_int"] = 4
    col.loc[col['emp_length'] == '3 years', "emp_length_int"] = 3
    col.loc[col['emp_length'] == '2 years', "emp_length_int"] = 2
    col.loc[col['emp_length'] == '1 year', "emp_length_int"] = 1
    col.loc[col['emp_length'] == '< 1 year', "emp_length_int"] = 0.5
    col.loc[col['emp_length'] == 'n/a', "emp_length_int"] = 0

In [ ]:
X = loan[['installment','inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_pymnt',
       'total_pymnt_inv','total_rec_int',
       'tot_coll_amt', 'tot_cur_bal',
        'loan_condition_int']].copy()
X.head()

In [ ]:
Y = loan['int_rate']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
features = X.columns.values

features

In [ ]:
sc = MinMaxScaler(feature_range= (0,1))
X = pd.DataFrame(sc.fit_transform(X), columns= features)

In [ ]:
X.head(5)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
new_df=pd.Dataframe()
new_df=loan[['installment','inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_pymnt',
       'total_pymnt_inv','total_rec_int',
       'tot_coll_amt', 'tot_cur_bal',
        'loan_condition_int','int_rate']].copy()

### Using H2o to find the best model:

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
new_df.to_csv('final_data.csv')

In [ ]:
df = h2o.import_file('final_data.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df['target'] = df['int_rate'].asfactor()
df.head()

In [ ]:
df.describe()

In [ ]:
y = "target"
x = features.tolist()

In [ ]:
aml = H2OAutoML(max_models = 10, max_runtime_secs=120, seed = 1)
aml.train(x = x, y = y, training_frame= df)

In [ ]:
lb = aml.leaderboard
lb.head()